In [1]:
import syft as sy


kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node = sy.orchestra.launch(name="test-jobs", dev_mode=True, reset=True, create_producer=True, queue_port=5556, n_consumers=3)
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Logged into <test-jobs: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
x = sy.ActionObject.from_obj([1,2,3])
x_ptr = x.send(client)

In [4]:
@sy.syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    import random
    sleep(random.random()*5)
    print("done")
    return batch + 1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

In [6]:
@sy.syft_function()
def aggregate_job(job_results):
    print([(res, type(res))for res in job_results])
    results = [res for res in job_results]
    return sum(results)

SyftSuccess: Syft function 'aggregate_job' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [7]:
client.code.submit(aggregate_job)

SyftSuccess: User Code Submitted

In [8]:
@sy.syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    job_results = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        job_results.append(batch_job.result)
    print("starting aggregation")
    print("Done")
    return domain.launch_job(aggregate_job, job_results=job_results)

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [9]:
client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-jobs


SyftSuccess: Request 2fd03f6fc79743fe824670f22fdae2e4 changes applied

In [10]:
from time import sleep
job = client.code.process_all(x=x_ptr, blocking=False)
sleep(10)

1 <class 'int'>
2 <class 'int'>
3 <class 'int'>


23/11/23 15:29:13 FUNCTION LOG (365b6090fb354b90b2ca26100ffa09e2): Launching jobs


1 <class 'int'>
2 <class 'int'>


23/11/23 15:29:14 FUNCTION LOG (365b6090fb354b90b2ca26100ffa09e2): starting aggregation
23/11/23 15:29:14 FUNCTION LOG (365b6090fb354b90b2ca26100ffa09e2): Done


3 <class 'int'>


23/11/23 15:29:15 FUNCTION LOG (0f162a8f4e71462888d1b2f2f74e7d62): starting batch 2
23/11/23 15:29:15 FUNCTION LOG (4abb66af50564c30aa4b74e4531a16d1): starting batch 1
23/11/23 15:29:15 FUNCTION LOG (4abb66af50564c30aa4b74e4531a16d1): done
23/11/23 15:29:16 FUNCTION LOG (40a0e1b77ccf4b749a1638f87b598928): starting batch 3
23/11/23 15:29:16 FUNCTION LOG (40a0e1b77ccf4b749a1638f87b598928): done
23/11/23 15:29:17 FUNCTION LOG (0f162a8f4e71462888d1b2f2f74e7d62): done


ActionDataEmpty <None> <class 'syft.service.action.action_object.AnyActionObject'>
ActionDataEmpty <None> <class 'syft.service.action.action_object.AnyActionObject'>
ActionDataEmpty <None> <class 'syft.service.action.action_object.AnyActionObject'>


23/11/23 15:29:19 FUNCTION LOG (099b48335f2d4dd5b0ea7c5277b533c0): [(2, <class 'int'>), (3, <class 'int'>), (4, <class 'int'>)]


In [11]:
result = job.resolve.get().resolve.get()
result

9